In [1]:
%reload_ext autoreload
%autoreload 2
import sys; sys.path.append('..')
from functools import partial
from src.data_utils import get_dataset_imagenet
from jupyter_preproc import smoothen, probs_to_softacc, nonmono, thresh, easy, normalize, get_entropy, accuracy
import numpy as np
import time
import os
from src.simple_utils import load_pickle

rng = np.random.default_rng(5)

In [2]:
from itertools import product

In [3]:


# dnames = ['imagenet_v2', 'imagenet_r', 'imagenet_a', 'imagenet_val', 'imagenet_sketch', 'my_data']
dnames = ['imagenet_aug', 'imagenet_vid']
gnames = ['resnet50', 'densenet121', 'densenet169']

for dname, gname in product(dnames, gnames):
    G = load_pickle(f'../processed/imagenet/{gname}/{gname}_{dname}_alldata')
    ## get it into standard form (index ordering)
    for k in ['probs', 'freqs', 'labels', 'labels_err', 'points', 'points_err']:
        G[k] = np.moveaxis(G[k], 0, -1)
        print(f'G[{k}] shape: {G[k].shape}')

    outfname = f'../processed/imagenet/{gname}/{gname}_{dname}.npz'
    print('saving to:', outfname)
    start_time = time.time()
    with open(outfname, 'wb') as f:
        np.savez(f, **G)
    print('done: %f', (time.time() - start_time))

G[probs] shape: (7004, 1000, 40)
G[freqs] shape: (7004, 1000, 40)
G[labels] shape: (1000, 40)
G[labels_err] shape: (1000, 40)
G[points] shape: (7004, 40)
G[points_err] shape: (7004, 40)
saving to: ../processed/imagenet/resnet50/resnet50_imagenet_aug.npz
done: %f 9.33575177192688
G[probs] shape: (7004, 1000, 40)
G[freqs] shape: (7004, 1000, 40)
G[labels] shape: (1000, 40)
G[labels_err] shape: (1000, 40)
G[points] shape: (7004, 40)
G[points_err] shape: (7004, 40)
saving to: ../processed/imagenet/densenet121/densenet121_imagenet_aug.npz
done: %f 7.114560842514038
G[probs] shape: (7004, 1000, 40)
G[freqs] shape: (7004, 1000, 40)
G[labels] shape: (1000, 40)
G[labels_err] shape: (1000, 40)
G[points] shape: (7004, 40)
G[points_err] shape: (7004, 40)
saving to: ../processed/imagenet/densenet169/densenet169_imagenet_aug.npz
done: %f 7.531619071960449
G[probs] shape: (51826, 1000, 40)
G[freqs] shape: (51826, 1000, 40)
G[labels] shape: (1, 40)
G[labels_err] shape: (1, 40)
G[points] shape: (51826,

In [4]:
tname = 'imagenet'
# dnames = ['imagenet_v2', 'imagenet_r', 'imagenet_a', 'imagenet_val', 'imagenet_sketch', 'my_data']
dnames = ['imagenet_vid', 'imagenet_aug']
gnames = ['resnet50', 'densenet121', 'densenet169']

# smoothing parameters
num_gridpts = 50
smooth = True
sigma = 2.0

xkey = 'x'
Ykey = 'probs'

measures = {'ent': get_entropy}
funcs = [nonmono, thresh, easy]

for dname in dnames:
    dataset = get_dataset_imagenet(dname)
    ytest = dataset.targets
    measures['acc'] = partial(accuracy, ytest=ytest)
    
    for gname in gnames:
        print(f'{dname} {gname}')
        fname = f'../processed/{tname}/{gname}/{gname}_{dname}.npz'
        
        print('loading processed')
        G = np.load(fname)
        print('done\n')
        
        # G['xsoft'] = probs_to_softacc(G['probs'], ytest)
        
        print('smoothing')
        res = smoothen(G, xkey, Ykey)
        print('done\n')
        
        scores = dict()
        for meas_name, meas in measures.items():
            print(meas_name)
            m = meas(res[f'{Ykey}_smooth'])
            print(m.shape)
    
            for func in funcs:
                print(func.__name__)
                f_name = f'{meas_name}_{func.__name__}'
                s_vec = func(m)
                print(s_vec.shape)
                scores[f_name] = normalize(s_vec)
        
        os.makedirs(f'../scores/{tname}/{gname}/', exist_ok=True)
        outfname = f'../scores/{tname}/{gname}/{gname}_{dname}_scores.npz'
        
        print(f'saving to: {outfname}')                     
        with open(outfname, 'wb') as f:
            np.savez(f, **scores)
        print('done\n')

imagenet_vid resnet50
loading processed
done

smoothing
done

ent
(51826, 50)
nonmono
(51826,)
thresh
(51826,)
easy
(51826,)
acc


IndexError: shape mismatch: indexing arrays could not be broadcast together with shapes (51826,) (74005,) 